**Chapter 16 – Natural Language Processing with RNNs and Attention**

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Char-RNN

## Splitting a sequence into batches of shuffled windows

For example, let's split the sequence 0 to 14 into windows of length 5, each shifted by 2 (e.g.,`[0, 1, 2, 3, 4]`, `[2, 3, 4, 5, 6]`, etc.), then shuffle them, and split them into inputs (the first 4 steps) and targets (the last 4 steps) (e.g., `[2, 3, 4, 5, 6]` would be split into `[[2, 3, 4, 5], [3, 4, 5, 6]]`), then create batches of 3 such input/target pairs:

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


## Loading the Data and Preparing the Dataset

In [5]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [6]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [7]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [8]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [9]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [10]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [11]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [12]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [13]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [14]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [17]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [18]:
dataset = dataset.prefetch(1)

In [19]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


## Creating and Training the Model

In [18]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [19]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)
np.save('history/shakespeare.npy', history.history)
model.save("models/shakespeare.h5")

Train for 31370 steps
Epoch 1/10
31370/31370 [==============================] - 6052s 193ms/step - loss: 1.4671TA - ETA: 6:04 - loss: 1.4 - ETA:
Epoch 2/10
31370/31370 [==============================] - 6052s 193ms/step - loss: 1.3614
Epoch 3/10
31370/31370 [==============================] - 6048s 193ms/step - loss: 1.34049:49 - - ETA: 9:3 - ETA: 9:33  -  - ETA: 8:50 - loss:  - ETA: 8:4  - ETA: 8:40 - lo - ETA: 8:33 - lo - ETA: - ETA: 8:17 -  - ETA: 8:15 - loss: 1.34 - ETA: 8:14  - ETA: 8:01 - ETA: 7:58 - loss: - ETA: 7:52 - los - ET - ETA: 7:41 - loss: 1 - ETA: 7:40 - loss: - ETA: 7:38 - loss: 1.3 - ETA: 7:33 - ETA: 7:24 - ETA: 7:16 - lo - ETA: 7:14 - - ETA: 7: - ETA: 7:03  - ETA: 7:01 - loss:  - ETA: 7:00 - loss: - - ETA: 6:4 - ETA: 6:40 - loss:  - ETA: 6:39 -  - ETA:  - ETA: 6:22 - loss: 1.34 - ETA: 6:22 - loss - ETA - ETA: 6:16 - loss: 1.340 - ETA: 6:16 - loss: 1.3 - ETA: 6:16 - loss: 1.340 - ETA: 6:15 - l - ETA: 6:13 - los - ETA: 6:06 - loss: 1.34 - ETA: - ETA: 3s - loss: 1.3 - ET

In [20]:
model = keras.models.load_model('models/shakespeare.h5')
history = np.load('history/shakespeare.npy',allow_pickle='TRUE').item()

## Using the Model to Generate Text

In [21]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [22]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'u'

In [23]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]],
      dtype=int64)

In [24]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [25]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

'u'

In [26]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [27]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

the state, the state and they are the belly
and the


In [28]:
print(complete_text("t", temperature=1))

to companio;
and they are spoke at jot, trivants al


In [29]:
print(complete_text("t", temperature=2))

tpeniom,
that?' tigl'd him frevils dear. ruli-hape-


## Stateful RNN

In [30]:
tf.random.set_seed(42)

In [31]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [32]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [33]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [34]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [35]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps

In [36]:
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])
np.save('history/stateful_rnn.npy', history.history)
model.save("models/stateful_rnn.h5")

Train for 313 steps
Epoch 1/50
313/313 [==============================] - 58s 186ms/step - loss: 2.61752s -
Epoch 2/50
313/313 [==============================] - 55s 177ms/step - loss: 2.19443s -  - ETA: 1s - los
Epoch 3/50
313/313 [==============================] - 57s 182ms/step - loss: 2.30811s - loss: 
Epoch 4/50
313/313 [==============================] - 56s 179ms/step - loss: 2.2082
Epoch 5/50
313/313 [==============================] - 56s 178ms/step - loss: 2.1247 9s - loss:   
Epoch 6/50
313/313 [==============================] - 55s 175ms/step - loss: 2.0825
Epoch 7/50
313/313 [==============================] - 58s 186ms/step - loss: 2.0365- ETA: 1s - los
Epoch 8/50
313/313 [==============================] - 60s 191ms/step - loss: 1.95382s -
Epoch 9/50
313/313 [==============================] - 57s 181ms/step - loss: 1.9183
Epoch 10/50
313/313 [==============================] - 55s 177ms/step - loss: 1.9242
Epoch 11/50
313/313 [==============================] - 55s 177ms/step 

In [37]:
model = keras.models.load_model('models/stateful_rnn.h5')
history = np.load('history/stateful_rnn.npy',allow_pickle='TRUE').item()

To use the model with different batch sizes, we need to create a stateless copy. We can get rid of dropout since it is only used during training:

In [38]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

To set the weights, we first need to build the model (so the weights get created):

In [39]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [40]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [41]:
tf.random.set_seed(42)

print(complete_text("t"))

think it,
did thou art to joy all. believe i not
th


# Sentiment Analysis

In [42]:
tf.random.set_seed(42)

You can load the IMDB dataset easily:

In [43]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 3s 0us/step


In [44]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [45]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1646592/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

In [46]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Shuffling and writing examples to C:\Users\com\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteX9NZL8\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\com\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteX9NZL8\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\com\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteX9NZL8\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\com\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [47]:
datasets.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [48]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [49]:
train_size, test_size

(25000, 25000)

In [50]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This is a big step down after the surprisingly enjoyable original. This sequel isn't nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best ...
Label: 0 = Negative

Review: Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian schoo ...
Label: 0 = Negative



In [51]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [52]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 60), dtype=string, numpy=
 array([[b'This', b'is', b'a', b'big', b'step', b'down', b'after', b'the',
         b'surprisingly', b'enjoyable', b'original', b'This', b'sequel',
         b"isn't", b'nearly', b'as', b'fun', b'as', b'part', b'one',
         b'and', b'it', b'instead', b'spends', b'too', b'much', b'time',
         b'on', b'plot', b'development', b'Tim', b'Thomerson', b'is',
         b'still', b'the', b'best', b'thing', b'about', b'this',
         b'series', b'but', b'his', b'wisecracking', b'is', b'toned',
         b'down', b'in', b'this', b'entry', b'The', b'performances',
         b'are', b'all', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
         b'<pad>', b'<pad>'],
        [b'Perhaps', b'because', b'I', b'was', b'so', b'young',
         b'innocent', b'and', b'BRAINWASHED', b'when', b'I', b'saw',
         b'it', b'this', b'movie', b'was', b'the', b'cause', b'of',
         b'many', b'sleepless', b'nights', b'for', b'me', b'I',
         b"haven'

In [53]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [54]:
vocabulary.most_common()[:3]

[(b'<pad>', 214741), (b'the', 61137), (b'a', 38564)]

In [55]:
len(vocabulary)

53893

In [56]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [57]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [58]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [59]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]], dtype=int64)>

In [60]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].repeat().batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [61]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[   22     7     2 ...     0     0     0]
 [ 1239    82     6 ...   418    28  4245]
 [ 4246     3     1 ...     0     0     0]
 ...
 [   22     7    23 ...     0     0     0]
 [ 1297  3744     7 ...     0     0     0]
 [10928 10687  4537 ...     0     0     0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1 1], shape=(32,), dtype=int64)


In [62]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)
np.save('history/sentiment_analysis.npy', history.history)
model.save("models/sentiment_analysis.h5")

Train for 781 steps
Epoch 1/5
541/781 [===================>..........] - ETA: 1:19:45 - loss: 0.6923 - accuracy: 0.656 - ETA: 40:10 - loss: 0.6972 - accuracy: 0.4531  - ETA: 26:59 - loss: 0.6952 - accuracy: 0.489 - ETA: 13:35 - loss: 0.6935 - accuracy: 0.531 - ETA: 9:07 - loss: 0.6938 - accuracy: 0.520 - ETA: 6:53 - loss: 0.6936 - accuracy: 0.52 - ETA: 5:32 - loss: 0.6934 - accuracy: 0.52 - ETA: 4:39 - loss: 0.6923 - accuracy: 0.54 - ETA: 4:01 - loss: 0.6942 - accuracy: 0.52 - ETA: 3:32 - loss: 0.6945 - accuracy: 0.51 - ETA: 3:09 - loss: 0.6944 - accuracy: 0.51 - ETA: 2:51 - loss: 0.6945 - accuracy: 0.50 - ETA: 2:37 - loss: 0.6943 - accuracy: 0.50 - ETA: 2:24 - loss: 0.6941 - accuracy: 0.50 - ETA: 2:14 - loss: 0.6944 - accuracy: 0.50 - ETA: 2:05 - loss: 0.6939 - accuracy: 0.50 - ETA: 1:58 - loss: 0.6938 - accuracy: 0.50 - ETA: 1:51 - loss: 0.6932 - accuracy: 0.51 - ETA: 1:45 - loss: 0.6929 - accuracy: 0.51 - ETA: 1:39 - loss: 0.6929 - accuracy: 0.51 - ETA: 1:35 - loss: 0.6932 - accurac

562/781 [====================>.........] - ETA: 18s - loss: 0.6274 - accuracy: 0.656 - ETA: 17s - loss: 0.4561 - accuracy: 0.804 - ETA: 17s - loss: 0.4399 - accuracy: 0.799 - ETA: 17s - loss: 0.4549 - accuracy: 0.787 - ETA: 17s - loss: 0.4830 - accuracy: 0.774 - ETA: 17s - loss: 0.4949 - accuracy: 0.765 - ETA: 17s - loss: 0.4857 - accuracy: 0.773 - ETA: 17s - loss: 0.4890 - accuracy: 0.768 - ETA: 17s - loss: 0.4849 - accuracy: 0.771 - ETA: 17s - loss: 0.4845 - accuracy: 0.773 - ETA: 17s - loss: 0.4832 - accuracy: 0.771 - ETA: 16s - loss: 0.4827 - accuracy: 0.773 - ETA: 16s - loss: 0.4756 - accuracy: 0.776 - ETA: 16s - loss: 0.4713 - accuracy: 0.778 - ETA: 16s - loss: 0.4746 - accuracy: 0.778 - ETA: 16s - loss: 0.4691 - accuracy: 0.782 - ETA: 16s - loss: 0.4666 - accuracy: 0.786 - ETA: 16s - loss: 0.4625 - accuracy: 0.787 - ETA: 16s - loss: 0.4650 - accuracy: 0.786 - ETA: 16s - loss: 0.4610 - accuracy: 0.788 - ETA: 16s - loss: 0.4557 - accuracy: 0.790 - ETA: 16s - loss: 0.4566 - accurac

781/781 [==============================] - ETA: 4s - loss: 0.3551 - accuracy: 0.85 - ETA: 4s - loss: 0.3546 - accuracy: 0.85 - ETA: 4s - loss: 0.3541 - accuracy: 0.85 - ETA: 4s - loss: 0.3540 - accuracy: 0.85 - ETA: 4s - loss: 0.3532 - accuracy: 0.85 - ETA: 4s - loss: 0.3531 - accuracy: 0.85 - ETA: 4s - loss: 0.3531 - accuracy: 0.85 - ETA: 4s - loss: 0.3529 - accuracy: 0.85 - ETA: 4s - loss: 0.3533 - accuracy: 0.85 - ETA: 4s - loss: 0.3533 - accuracy: 0.85 - ETA: 4s - loss: 0.3531 - accuracy: 0.85 - ETA: 4s - loss: 0.3531 - accuracy: 0.85 - ETA: 4s - loss: 0.3526 - accuracy: 0.85 - ETA: 4s - loss: 0.3527 - accuracy: 0.85 - ETA: 3s - loss: 0.3526 - accuracy: 0.85 - ETA: 3s - loss: 0.3531 - accuracy: 0.85 - ETA: 3s - loss: 0.3534 - accuracy: 0.85 - ETA: 3s - loss: 0.3542 - accuracy: 0.85 - ETA: 3s - loss: 0.3544 - accuracy: 0.85 - ETA: 3s - loss: 0.3548 - accuracy: 0.85 - ETA: 3s - loss: 0.3553 - accuracy: 0.85 - ETA: 3s - loss: 0.3550 - accuracy: 0.85 - ETA: 3s - loss: 0.3553 - accuracy

562/781 [====================>.........] - ETA: 17s - loss: 0.2390 - accuracy: 0.937 - ETA: 17s - loss: 0.1717 - accuracy: 0.960 - ETA: 17s - loss: 0.1643 - accuracy: 0.955 - ETA: 17s - loss: 0.1548 - accuracy: 0.953 - ETA: 17s - loss: 0.1608 - accuracy: 0.951 - ETA: 17s - loss: 0.1647 - accuracy: 0.949 - ETA: 17s - loss: 0.1826 - accuracy: 0.944 - ETA: 17s - loss: 0.1686 - accuracy: 0.947 - ETA: 17s - loss: 0.1648 - accuracy: 0.948 - ETA: 17s - loss: 0.1644 - accuracy: 0.946 - ETA: 16s - loss: 0.1597 - accuracy: 0.947 - ETA: 16s - loss: 0.1595 - accuracy: 0.949 - ETA: 16s - loss: 0.1536 - accuracy: 0.952 - ETA: 16s - loss: 0.1543 - accuracy: 0.953 - ETA: 16s - loss: 0.1563 - accuracy: 0.952 - ETA: 16s - loss: 0.1530 - accuracy: 0.953 - ETA: 16s - loss: 0.1498 - accuracy: 0.954 - ETA: 16s - loss: 0.1475 - accuracy: 0.955 - ETA: 16s - loss: 0.1478 - accuracy: 0.954 - ETA: 16s - loss: 0.1478 - accuracy: 0.953 - ETA: 16s - loss: 0.1475 - accuracy: 0.954 - ETA: 16s - loss: 0.1476 - accurac

781/781 [==============================] - ETA: 4s - loss: 0.1244 - accuracy: 0.95 - ETA: 4s - loss: 0.1240 - accuracy: 0.95 - ETA: 4s - loss: 0.1236 - accuracy: 0.95 - ETA: 4s - loss: 0.1231 - accuracy: 0.95 - ETA: 4s - loss: 0.1229 - accuracy: 0.95 - ETA: 4s - loss: 0.1234 - accuracy: 0.95 - ETA: 4s - loss: 0.1235 - accuracy: 0.95 - ETA: 4s - loss: 0.1234 - accuracy: 0.95 - ETA: 4s - loss: 0.1239 - accuracy: 0.95 - ETA: 4s - loss: 0.1246 - accuracy: 0.95 - ETA: 4s - loss: 0.1251 - accuracy: 0.95 - ETA: 4s - loss: 0.1252 - accuracy: 0.95 - ETA: 4s - loss: 0.1250 - accuracy: 0.95 - ETA: 4s - loss: 0.1253 - accuracy: 0.95 - ETA: 3s - loss: 0.1257 - accuracy: 0.95 - ETA: 3s - loss: 0.1257 - accuracy: 0.95 - ETA: 3s - loss: 0.1259 - accuracy: 0.95 - ETA: 3s - loss: 0.1263 - accuracy: 0.95 - ETA: 3s - loss: 0.1266 - accuracy: 0.95 - ETA: 3s - loss: 0.1272 - accuracy: 0.95 - ETA: 3s - loss: 0.1274 - accuracy: 0.95 - ETA: 3s - loss: 0.1272 - accuracy: 0.95 - ETA: 3s - loss: 0.1276 - accuracy

In [67]:
#dependencies = {'accuracy': accuracy}
model = keras.models.load_model('models/sentiment_analysis.h5')
history = np.load('history/sentiment_analysis.npy',allow_pickle='TRUE').item()

Or using manual masking:

In [68]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [69]:
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)
np.save('history/manual_masking.npy', history.history)
model.save("models/manual_masking.h5")

Train for 781 steps
Epoch 1/5
535/781 [===================>..........] - ETA: 1:15:07 - loss: 0.6919 - accuracy: 0.531 - ETA: 25:12 - loss: 0.6956 - accuracy: 0.4479  - ETA: 15:13 - loss: 0.6948 - accuracy: 0.462 - ETA: 10:56 - loss: 0.6942 - accuracy: 0.482 - ETA: 7:43 - loss: 0.6940 - accuracy: 0.475 - ETA: 5:59 - loss: 0.6941 - accuracy: 0.47 - ETA: 4:54 - loss: 0.6937 - accuracy: 0.48 - ETA: 4:09 - loss: 0.6932 - accuracy: 0.50 - ETA: 3:37 - loss: 0.6950 - accuracy: 0.48 - ETA: 3:12 - loss: 0.6955 - accuracy: 0.48 - ETA: 2:52 - loss: 0.6948 - accuracy: 0.49 - ETA: 2:37 - loss: 0.6948 - accuracy: 0.48 - ETA: 2:24 - loss: 0.6943 - accuracy: 0.49 - ETA: 2:13 - loss: 0.6944 - accuracy: 0.49 - ETA: 2:07 - loss: 0.6947 - accuracy: 0.48 - ETA: 1:58 - loss: 0.6944 - accuracy: 0.49 - ETA: 1:51 - loss: 0.6943 - accuracy: 0.49 - ETA: 1:45 - loss: 0.6936 - accuracy: 0.49 - ETA: 1:39 - loss: 0.6933 - accuracy: 0.49 - ETA: 1:34 - loss: 0.6931 - accuracy: 0.50 - ETA: 1:30 - loss: 0.6934 - accurac

558/781 [====================>.........] - ETA: 21s - loss: 0.6260 - accuracy: 0.656 - ETA: 19s - loss: 0.4490 - accuracy: 0.804 - ETA: 19s - loss: 0.4358 - accuracy: 0.807 - ETA: 19s - loss: 0.4381 - accuracy: 0.802 - ETA: 19s - loss: 0.4471 - accuracy: 0.792 - ETA: 19s - loss: 0.4691 - accuracy: 0.778 - ETA: 19s - loss: 0.4847 - accuracy: 0.769 - ETA: 18s - loss: 0.4757 - accuracy: 0.781 - ETA: 18s - loss: 0.4770 - accuracy: 0.777 - ETA: 18s - loss: 0.4725 - accuracy: 0.780 - ETA: 17s - loss: 0.4721 - accuracy: 0.781 - ETA: 17s - loss: 0.4743 - accuracy: 0.781 - ETA: 17s - loss: 0.4726 - accuracy: 0.783 - ETA: 17s - loss: 0.4653 - accuracy: 0.785 - ETA: 17s - loss: 0.4617 - accuracy: 0.790 - ETA: 16s - loss: 0.4664 - accuracy: 0.791 - ETA: 16s - loss: 0.4606 - accuracy: 0.792 - ETA: 16s - loss: 0.4597 - accuracy: 0.793 - ETA: 16s - loss: 0.4567 - accuracy: 0.795 - ETA: 16s - loss: 0.4529 - accuracy: 0.796 - ETA: 16s - loss: 0.4576 - accuracy: 0.794 - ETA: 16s - loss: 0.4524 - accurac

781/781 [==============================] - ETA: 4s - loss: 0.3535 - accuracy: 0.85 - ETA: 4s - loss: 0.3536 - accuracy: 0.85 - ETA: 4s - loss: 0.3534 - accuracy: 0.85 - ETA: 4s - loss: 0.3527 - accuracy: 0.85 - ETA: 4s - loss: 0.3525 - accuracy: 0.85 - ETA: 4s - loss: 0.3518 - accuracy: 0.85 - ETA: 4s - loss: 0.3517 - accuracy: 0.85 - ETA: 4s - loss: 0.3518 - accuracy: 0.85 - ETA: 4s - loss: 0.3516 - accuracy: 0.85 - ETA: 4s - loss: 0.3523 - accuracy: 0.85 - ETA: 4s - loss: 0.3520 - accuracy: 0.85 - ETA: 4s - loss: 0.3521 - accuracy: 0.85 - ETA: 4s - loss: 0.3522 - accuracy: 0.85 - ETA: 3s - loss: 0.3519 - accuracy: 0.85 - ETA: 3s - loss: 0.3519 - accuracy: 0.85 - ETA: 3s - loss: 0.3515 - accuracy: 0.85 - ETA: 3s - loss: 0.3515 - accuracy: 0.85 - ETA: 3s - loss: 0.3517 - accuracy: 0.85 - ETA: 3s - loss: 0.3521 - accuracy: 0.85 - ETA: 3s - loss: 0.3524 - accuracy: 0.85 - ETA: 3s - loss: 0.3525 - accuracy: 0.85 - ETA: 3s - loss: 0.3527 - accuracy: 0.85 - ETA: 3s - loss: 0.3527 - accuracy

564/781 [====================>.........] - ETA: 18s - loss: 0.2124 - accuracy: 0.937 - ETA: 17s - loss: 0.1654 - accuracy: 0.960 - ETA: 17s - loss: 0.1503 - accuracy: 0.964 - ETA: 17s - loss: 0.1527 - accuracy: 0.959 - ETA: 17s - loss: 0.1649 - accuracy: 0.954 - ETA: 17s - loss: 0.1652 - accuracy: 0.947 - ETA: 17s - loss: 0.1809 - accuracy: 0.940 - ETA: 17s - loss: 0.1756 - accuracy: 0.941 - ETA: 17s - loss: 0.1798 - accuracy: 0.938 - ETA: 17s - loss: 0.1796 - accuracy: 0.939 - ETA: 17s - loss: 0.1783 - accuracy: 0.940 - ETA: 16s - loss: 0.1752 - accuracy: 0.943 - ETA: 16s - loss: 0.1726 - accuracy: 0.945 - ETA: 16s - loss: 0.1701 - accuracy: 0.947 - ETA: 16s - loss: 0.1726 - accuracy: 0.946 - ETA: 16s - loss: 0.1651 - accuracy: 0.949 - ETA: 16s - loss: 0.1640 - accuracy: 0.948 - ETA: 16s - loss: 0.1594 - accuracy: 0.950 - ETA: 16s - loss: 0.1608 - accuracy: 0.950 - ETA: 16s - loss: 0.1626 - accuracy: 0.949 - ETA: 16s - loss: 0.1598 - accuracy: 0.951 - ETA: 16s - loss: 0.1578 - accurac

781/781 [==============================] - ETA: 4s - loss: 0.1300 - accuracy: 0.95 - ETA: 4s - loss: 0.1299 - accuracy: 0.95 - ETA: 4s - loss: 0.1297 - accuracy: 0.95 - ETA: 4s - loss: 0.1293 - accuracy: 0.95 - ETA: 4s - loss: 0.1294 - accuracy: 0.95 - ETA: 4s - loss: 0.1289 - accuracy: 0.95 - ETA: 4s - loss: 0.1288 - accuracy: 0.95 - ETA: 4s - loss: 0.1294 - accuracy: 0.95 - ETA: 4s - loss: 0.1293 - accuracy: 0.95 - ETA: 4s - loss: 0.1292 - accuracy: 0.95 - ETA: 4s - loss: 0.1289 - accuracy: 0.95 - ETA: 4s - loss: 0.1290 - accuracy: 0.95 - ETA: 4s - loss: 0.1287 - accuracy: 0.95 - ETA: 3s - loss: 0.1286 - accuracy: 0.95 - ETA: 3s - loss: 0.1286 - accuracy: 0.95 - ETA: 3s - loss: 0.1292 - accuracy: 0.95 - ETA: 3s - loss: 0.1293 - accuracy: 0.95 - ETA: 3s - loss: 0.1294 - accuracy: 0.95 - ETA: 3s - loss: 0.1298 - accuracy: 0.95 - ETA: 3s - loss: 0.1299 - accuracy: 0.95 - ETA: 3s - loss: 0.1295 - accuracy: 0.95 - ETA: 3s - loss: 0.1296 - accuracy: 0.95 - ETA: 3s - loss: 0.1300 - accuracy

In [19]:
dependencies = {}#'accuracy': accuracy}
model = keras.models.load_model('models/manual_masking.h5', custom_objects=dependencies)
history = np.load('history/manual_masking.npy',allow_pickle='TRUE').item()

## Reusing Pretrained Embeddings

In [25]:
tf.random.set_seed(42)

In [26]:
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [23]:
!pip install tensorflow_hub

In [32]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [33]:
embeddings

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.16589954,  0.0254965 ,  0.1574857 ,  0.17688066,  0.02911299,
        -0.03092718,  0.19445257, -0.05709129, -0.08631689, -0.04391516,
         0.13032274,  0.10905275, -0.08515751,  0.01056632, -0.17220995,
        -0.17925954,  0.19556305,  0.0802278 , -0.03247919, -0.49176937,
        -0.07767699, -0.03160921, -0.13952136,  0.05959712,  0.06858718,
         0.22386682, -0.16653948,  0.19412343, -0.05491862,  0.10997339,
        -0.15811177, -0.02576607, -0.07910853, -0.258499  , -0.04206644,
        -0.20052543,  0.1705603 , -0.15314153,  0.0039225 , -0.28694248,
         0.02468278,  0.11069503,  0.03733957,  0.01433943, -0.11048374,
         0.11931834, -0.11552787, -0.11110869,  0.02384969, -0.07074881],
       [ 0.1437864 ,  0.08291595,  0.10897306,  0.04464385, -0.03630389,
        -0.12605834,  0.20263346,  0.12862863, -0.07873426, -0.01195358,
         0.0020956 , -0.03080653, -0.08019945, -0.18797135, -0.11973457,
 

In [27]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [28]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\saved_model.pb
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\assets\tokens.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.data-00000-of-00001
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.index


In [29]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)

In [30]:
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=10)
np.save('history/pretrained_embedding.npy', history.history)
model.save("models/pretrained_embedding.h5")

Train for 781 steps
Epoch 1/10
781/781 [==============================] - ETA: 44:15 - loss: 0.7482 - accuracy: 0.437 - ETA: 8:56 - loss: 0.7029 - accuracy: 0.525 - ETA: 3:45 - loss: 0.6877 - accuracy: 0.54 - ETA: 2:22 - loss: 0.6841 - accuracy: 0.55 - ETA: 1:45 - loss: 0.6794 - accuracy: 0.57 - ETA: 1:23 - loss: 0.6732 - accuracy: 0.59 - ETA: 1:07 - loss: 0.6705 - accuracy: 0.59 - ETA: 57s - loss: 0.6649 - accuracy: 0.6100 - ETA: 50s - loss: 0.6631 - accuracy: 0.615 - ETA: 44s - loss: 0.6594 - accuracy: 0.621 - ETA: 40s - loss: 0.6591 - accuracy: 0.623 - ETA: 36s - loss: 0.6549 - accuracy: 0.629 - ETA: 33s - loss: 0.6523 - accuracy: 0.631 - ETA: 30s - loss: 0.6487 - accuracy: 0.635 - ETA: 28s - loss: 0.6446 - accuracy: 0.641 - ETA: 26s - loss: 0.6410 - accuracy: 0.646 - ETA: 24s - loss: 0.6379 - accuracy: 0.650 - ETA: 23s - loss: 0.6345 - accuracy: 0.655 - ETA: 21s - loss: 0.6322 - accuracy: 0.658 - ETA: 20s - loss: 0.6302 - accuracy: 0.658 - ETA: 19s - loss: 0.6299 - accuracy: 0.658 

781/781 [==============================] - ETA: 4s - loss: 0.5799 - accuracy: 0.68 - ETA: 5s - loss: 0.5053 - accuracy: 0.75 - ETA: 5s - loss: 0.5093 - accuracy: 0.75 - ETA: 5s - loss: 0.5074 - accuracy: 0.74 - ETA: 5s - loss: 0.5004 - accuracy: 0.75 - ETA: 5s - loss: 0.5107 - accuracy: 0.75 - ETA: 5s - loss: 0.5146 - accuracy: 0.75 - ETA: 5s - loss: 0.5133 - accuracy: 0.75 - ETA: 5s - loss: 0.5282 - accuracy: 0.74 - ETA: 5s - loss: 0.5273 - accuracy: 0.74 - ETA: 5s - loss: 0.5319 - accuracy: 0.74 - ETA: 5s - loss: 0.5320 - accuracy: 0.73 - ETA: 5s - loss: 0.5326 - accuracy: 0.73 - ETA: 4s - loss: 0.5293 - accuracy: 0.73 - ETA: 4s - loss: 0.5286 - accuracy: 0.74 - ETA: 4s - loss: 0.5294 - accuracy: 0.73 - ETA: 4s - loss: 0.5280 - accuracy: 0.73 - ETA: 4s - loss: 0.5288 - accuracy: 0.73 - ETA: 4s - loss: 0.5280 - accuracy: 0.73 - ETA: 4s - loss: 0.5292 - accuracy: 0.73 - ETA: 4s - loss: 0.5296 - accuracy: 0.73 - ETA: 4s - loss: 0.5308 - accuracy: 0.73 - ETA: 4s - loss: 0.5279 - accuracy

781/781 [==============================] - ETA: 6s - loss: 0.6044 - accuracy: 0.68 - ETA: 6s - loss: 0.5112 - accuracy: 0.75 - ETA: 5s - loss: 0.5043 - accuracy: 0.76 - ETA: 5s - loss: 0.5044 - accuracy: 0.75 - ETA: 5s - loss: 0.4952 - accuracy: 0.76 - ETA: 5s - loss: 0.5045 - accuracy: 0.75 - ETA: 5s - loss: 0.5101 - accuracy: 0.75 - ETA: 5s - loss: 0.5125 - accuracy: 0.75 - ETA: 5s - loss: 0.5233 - accuracy: 0.75 - ETA: 5s - loss: 0.5231 - accuracy: 0.74 - ETA: 5s - loss: 0.5253 - accuracy: 0.74 - ETA: 5s - loss: 0.5272 - accuracy: 0.74 - ETA: 5s - loss: 0.5279 - accuracy: 0.74 - ETA: 5s - loss: 0.5241 - accuracy: 0.74 - ETA: 4s - loss: 0.5238 - accuracy: 0.74 - ETA: 4s - loss: 0.5243 - accuracy: 0.74 - ETA: 4s - loss: 0.5223 - accuracy: 0.74 - ETA: 4s - loss: 0.5225 - accuracy: 0.74 - ETA: 4s - loss: 0.5220 - accuracy: 0.74 - ETA: 4s - loss: 0.5221 - accuracy: 0.74 - ETA: 4s - loss: 0.5234 - accuracy: 0.74 - ETA: 4s - loss: 0.5240 - accuracy: 0.74 - ETA: 4s - loss: 0.5222 - accuracy

781/781 [==============================] - ETA: 5s - loss: 0.6226 - accuracy: 0.68 - ETA: 5s - loss: 0.4950 - accuracy: 0.78 - ETA: 5s - loss: 0.5017 - accuracy: 0.77 - ETA: 5s - loss: 0.5037 - accuracy: 0.76 - ETA: 5s - loss: 0.4963 - accuracy: 0.77 - ETA: 5s - loss: 0.5050 - accuracy: 0.76 - ETA: 5s - loss: 0.5060 - accuracy: 0.76 - ETA: 5s - loss: 0.5056 - accuracy: 0.76 - ETA: 5s - loss: 0.5199 - accuracy: 0.75 - ETA: 5s - loss: 0.5197 - accuracy: 0.75 - ETA: 5s - loss: 0.5231 - accuracy: 0.75 - ETA: 5s - loss: 0.5228 - accuracy: 0.75 - ETA: 5s - loss: 0.5229 - accuracy: 0.74 - ETA: 5s - loss: 0.5216 - accuracy: 0.74 - ETA: 4s - loss: 0.5211 - accuracy: 0.75 - ETA: 4s - loss: 0.5202 - accuracy: 0.74 - ETA: 4s - loss: 0.5181 - accuracy: 0.75 - ETA: 4s - loss: 0.5185 - accuracy: 0.75 - ETA: 4s - loss: 0.5165 - accuracy: 0.75 - ETA: 4s - loss: 0.5176 - accuracy: 0.74 - ETA: 4s - loss: 0.5193 - accuracy: 0.74 - ETA: 4s - loss: 0.5194 - accuracy: 0.74 - ETA: 4s - loss: 0.5189 - accuracy

781/781 [==============================] - ETA: 5s - loss: 0.5376 - accuracy: 0.65 - ETA: 5s - loss: 0.5033 - accuracy: 0.77 - ETA: 5s - loss: 0.5018 - accuracy: 0.77 - ETA: 5s - loss: 0.5001 - accuracy: 0.76 - ETA: 5s - loss: 0.4880 - accuracy: 0.77 - ETA: 5s - loss: 0.5019 - accuracy: 0.76 - ETA: 5s - loss: 0.5032 - accuracy: 0.76 - ETA: 5s - loss: 0.5062 - accuracy: 0.76 - ETA: 5s - loss: 0.5145 - accuracy: 0.75 - ETA: 5s - loss: 0.5155 - accuracy: 0.75 - ETA: 5s - loss: 0.5190 - accuracy: 0.75 - ETA: 5s - loss: 0.5194 - accuracy: 0.75 - ETA: 5s - loss: 0.5209 - accuracy: 0.74 - ETA: 5s - loss: 0.5194 - accuracy: 0.74 - ETA: 5s - loss: 0.5186 - accuracy: 0.74 - ETA: 5s - loss: 0.5154 - accuracy: 0.75 - ETA: 4s - loss: 0.5158 - accuracy: 0.74 - ETA: 4s - loss: 0.5152 - accuracy: 0.75 - ETA: 4s - loss: 0.5140 - accuracy: 0.75 - ETA: 4s - loss: 0.5140 - accuracy: 0.75 - ETA: 4s - loss: 0.5148 - accuracy: 0.74 - ETA: 4s - loss: 0.5148 - accuracy: 0.74 - ETA: 4s - loss: 0.5162 - accuracy

781/781 [==============================] - ETA: 6s - loss: 0.4812 - accuracy: 0.71 - ETA: 6s - loss: 0.4995 - accuracy: 0.78 - ETA: 5s - loss: 0.4953 - accuracy: 0.77 - ETA: 5s - loss: 0.4979 - accuracy: 0.76 - ETA: 5s - loss: 0.4845 - accuracy: 0.76 - ETA: 5s - loss: 0.4938 - accuracy: 0.76 - ETA: 5s - loss: 0.4961 - accuracy: 0.76 - ETA: 5s - loss: 0.5012 - accuracy: 0.76 - ETA: 5s - loss: 0.5101 - accuracy: 0.75 - ETA: 5s - loss: 0.5109 - accuracy: 0.75 - ETA: 5s - loss: 0.5176 - accuracy: 0.75 - ETA: 5s - loss: 0.5139 - accuracy: 0.75 - ETA: 5s - loss: 0.5158 - accuracy: 0.75 - ETA: 5s - loss: 0.5161 - accuracy: 0.74 - ETA: 5s - loss: 0.5150 - accuracy: 0.75 - ETA: 5s - loss: 0.5124 - accuracy: 0.75 - ETA: 5s - loss: 0.5113 - accuracy: 0.75 - ETA: 4s - loss: 0.5097 - accuracy: 0.75 - ETA: 4s - loss: 0.5093 - accuracy: 0.75 - ETA: 4s - loss: 0.5094 - accuracy: 0.75 - ETA: 4s - loss: 0.5110 - accuracy: 0.75 - ETA: 4s - loss: 0.5115 - accuracy: 0.75 - ETA: 4s - loss: 0.5126 - accuracy

781/781 [==============================] - ETA: 8s - loss: 0.4103 - accuracy: 0.75 - ETA: 6s - loss: 0.4936 - accuracy: 0.78 - ETA: 5s - loss: 0.4880 - accuracy: 0.77 - ETA: 5s - loss: 0.4932 - accuracy: 0.76 - ETA: 5s - loss: 0.4830 - accuracy: 0.77 - ETA: 5s - loss: 0.4893 - accuracy: 0.77 - ETA: 5s - loss: 0.4940 - accuracy: 0.76 - ETA: 5s - loss: 0.4994 - accuracy: 0.76 - ETA: 5s - loss: 0.5068 - accuracy: 0.76 - ETA: 5s - loss: 0.5070 - accuracy: 0.75 - ETA: 5s - loss: 0.5152 - accuracy: 0.75 - ETA: 5s - loss: 0.5121 - accuracy: 0.75 - ETA: 5s - loss: 0.5150 - accuracy: 0.75 - ETA: 5s - loss: 0.5131 - accuracy: 0.75 - ETA: 5s - loss: 0.5118 - accuracy: 0.75 - ETA: 5s - loss: 0.5092 - accuracy: 0.75 - ETA: 4s - loss: 0.5079 - accuracy: 0.75 - ETA: 4s - loss: 0.5070 - accuracy: 0.75 - ETA: 4s - loss: 0.5065 - accuracy: 0.75 - ETA: 4s - loss: 0.5067 - accuracy: 0.75 - ETA: 4s - loss: 0.5087 - accuracy: 0.75 - ETA: 4s - loss: 0.5083 - accuracy: 0.75 - ETA: 4s - loss: 0.5098 - accuracy

781/781 [==============================] - ETA: 6s - loss: 0.3923 - accuracy: 0.84 - ETA: 6s - loss: 0.4935 - accuracy: 0.78 - ETA: 5s - loss: 0.4879 - accuracy: 0.77 - ETA: 5s - loss: 0.4861 - accuracy: 0.76 - ETA: 5s - loss: 0.4797 - accuracy: 0.77 - ETA: 5s - loss: 0.4860 - accuracy: 0.77 - ETA: 5s - loss: 0.4938 - accuracy: 0.77 - ETA: 5s - loss: 0.4974 - accuracy: 0.76 - ETA: 5s - loss: 0.5055 - accuracy: 0.76 - ETA: 5s - loss: 0.5065 - accuracy: 0.75 - ETA: 5s - loss: 0.5120 - accuracy: 0.75 - ETA: 5s - loss: 0.5099 - accuracy: 0.75 - ETA: 5s - loss: 0.5124 - accuracy: 0.75 - ETA: 5s - loss: 0.5113 - accuracy: 0.75 - ETA: 5s - loss: 0.5101 - accuracy: 0.75 - ETA: 5s - loss: 0.5065 - accuracy: 0.75 - ETA: 4s - loss: 0.5057 - accuracy: 0.75 - ETA: 4s - loss: 0.5049 - accuracy: 0.75 - ETA: 4s - loss: 0.5045 - accuracy: 0.75 - ETA: 4s - loss: 0.5043 - accuracy: 0.75 - ETA: 4s - loss: 0.5055 - accuracy: 0.75 - ETA: 4s - loss: 0.5064 - accuracy: 0.75 - ETA: 4s - loss: 0.5069 - accuracy

781/781 [==============================] - ETA: 7s - loss: 0.4476 - accuracy: 0.78 - ETA: 5s - loss: 0.4986 - accuracy: 0.78 - ETA: 5s - loss: 0.4841 - accuracy: 0.77 - ETA: 5s - loss: 0.4891 - accuracy: 0.75 - ETA: 5s - loss: 0.4804 - accuracy: 0.76 - ETA: 5s - loss: 0.4857 - accuracy: 0.77 - ETA: 5s - loss: 0.4905 - accuracy: 0.76 - ETA: 5s - loss: 0.4937 - accuracy: 0.76 - ETA: 5s - loss: 0.5019 - accuracy: 0.76 - ETA: 5s - loss: 0.5037 - accuracy: 0.75 - ETA: 5s - loss: 0.5108 - accuracy: 0.75 - ETA: 5s - loss: 0.5072 - accuracy: 0.75 - ETA: 5s - loss: 0.5094 - accuracy: 0.75 - ETA: 5s - loss: 0.5098 - accuracy: 0.75 - ETA: 5s - loss: 0.5083 - accuracy: 0.75 - ETA: 4s - loss: 0.5045 - accuracy: 0.75 - ETA: 4s - loss: 0.5045 - accuracy: 0.75 - ETA: 4s - loss: 0.5040 - accuracy: 0.75 - ETA: 4s - loss: 0.5025 - accuracy: 0.75 - ETA: 4s - loss: 0.5021 - accuracy: 0.75 - ETA: 4s - loss: 0.5038 - accuracy: 0.75 - ETA: 4s - loss: 0.5037 - accuracy: 0.75 - ETA: 4s - loss: 0.5061 - accuracy

781/781 [==============================] - ETA: 7s - loss: 0.4432 - accuracy: 0.81 - ETA: 5s - loss: 0.4840 - accuracy: 0.80 - ETA: 5s - loss: 0.4787 - accuracy: 0.78 - ETA: 5s - loss: 0.4858 - accuracy: 0.76 - ETA: 5s - loss: 0.4754 - accuracy: 0.77 - ETA: 5s - loss: 0.4835 - accuracy: 0.77 - ETA: 5s - loss: 0.4894 - accuracy: 0.77 - ETA: 5s - loss: 0.4901 - accuracy: 0.77 - ETA: 5s - loss: 0.5018 - accuracy: 0.76 - ETA: 5s - loss: 0.5001 - accuracy: 0.76 - ETA: 5s - loss: 0.5045 - accuracy: 0.76 - ETA: 5s - loss: 0.5038 - accuracy: 0.76 - ETA: 5s - loss: 0.5058 - accuracy: 0.75 - ETA: 4s - loss: 0.5027 - accuracy: 0.76 - ETA: 4s - loss: 0.5018 - accuracy: 0.76 - ETA: 4s - loss: 0.5024 - accuracy: 0.76 - ETA: 4s - loss: 0.5001 - accuracy: 0.76 - ETA: 4s - loss: 0.4994 - accuracy: 0.76 - ETA: 4s - loss: 0.4990 - accuracy: 0.76 - ETA: 4s - loss: 0.5002 - accuracy: 0.76 - ETA: 4s - loss: 0.5002 - accuracy: 0.76 - ETA: 4s - loss: 0.5018 - accuracy: 0.75 - ETA: 4s - loss: 0.4999 - accuracy

In [34]:
dependencies = {'KerasLayer':hub.KerasLayer}#'accuracy': accuracy}
model = keras.models.load_model('models/pretrained_embedding.h5', custom_objects=dependencies)
history = np.load('history/pretrained_embedding.npy',allow_pickle='TRUE').item()

## Automatic Translation

In [35]:
tf.random.set_seed(42)

In [36]:
vocab_size = 100
embed_size = 10

In [37]:
!pip install tensorflow_addons

In [37]:
import tensorflow_addons as tfa

E:\ProgramData\Anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.1.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


AttributeError: module 'tensorflow_core._api.v2.random' has no attribute 'Generator'

In [39]:
tf.__version__

'2.1.0'

In [38]:
tfa.__version__

NameError: name 'tfa' is not defined

In [39]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

AttributeError: module 'tensorflow_core._api.v2.random' has no attribute 'Generator'

In [35]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [36]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

ValueError: Error when checking input: expected keras_layer_input to have 1 dimensions, but got array with shape (1000, 10)

### Bidirectional Recurrent Layers

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()

### Positional Encoding

In [ ]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1 # max_dims must be even
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

In [ ]:
max_steps = 201
max_dims = 512
pos_emb = PositionalEncoding(max_steps, max_dims)
PE = pos_emb(np.zeros((1, max_steps, max_dims), np.float32))[0].numpy()

In [ ]:
i1, i2, crop_i = 100, 101, 150
p1, p2, p3 = 22, 60, 35
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(9, 5))
ax1.plot([p1, p1], [-1, 1], "k--", label="$p = {}$".format(p1))
ax1.plot([p2, p2], [-1, 1], "k--", label="$p = {}$".format(p2), alpha=0.5)
ax1.plot(p3, PE[p3, i1], "bx", label="$p = {}$".format(p3))
ax1.plot(PE[:,i1], "b-", label="$i = {}$".format(i1))
ax1.plot(PE[:,i2], "r-", label="$i = {}$".format(i2))
ax1.plot([p1, p2], [PE[p1, i1], PE[p2, i1]], "bo")
ax1.plot([p1, p2], [PE[p1, i2], PE[p2, i2]], "ro")
ax1.legend(loc="center right", fontsize=14, framealpha=0.95)
ax1.set_ylabel("$P_{(p,i)}$", rotation=0, fontsize=16)
ax1.grid(True, alpha=0.3)
ax1.hlines(0, 0, max_steps - 1, color="k", linewidth=1, alpha=0.3)
ax1.axis([0, max_steps - 1, -1, 1])
ax2.imshow(PE.T[:crop_i], cmap="gray", interpolation="bilinear", aspect="auto")
ax2.hlines(i1, 0, max_steps - 1, color="b")
cheat = 2 # need to raise the red line a bit, or else it hides the blue one
ax2.hlines(i2+cheat, 0, max_steps - 1, color="r")
ax2.plot([p1, p1], [0, crop_i], "k--")
ax2.plot([p2, p2], [0, crop_i], "k--", alpha=0.5)
ax2.plot([p1, p2], [i2+cheat, i2+cheat], "ro")
ax2.plot([p1, p2], [i1, i1], "bo")
ax2.axis([0, max_steps - 1, 0, crop_i])
ax2.set_xlabel("$p$", fontsize=16)
ax2.set_ylabel("$i$", rotation=0, fontsize=16)
plt.show()

In [ ]:
embed_size = 512; max_steps = 500; vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

Here is a (very) simplified Transformer (the actual architecture has skip connections, layer norm, dense nets, and most importantly it uses Multi-Head Attention instead of regular Attention):

In [ ]:
Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(
    keras.layers.Dense(vocab_size, activation="softmax"))(Z)

Here's a basic implementation of the `MultiHeadAttention` layer. One will likely be added to `keras.layers` in the near future. Note that `Conv1D` layers with `kernel_size=1` (and the default `padding="valid"` and `strides=1`) is equivalent to a `TimeDistributed(Dense(...))` layer.

In [ ]:
K = keras.backend

class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, n_heads, causal=False, use_scale=False, **kwargs):
        self.n_heads = n_heads
        self.causal = causal
        self.use_scale = use_scale
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.dims = batch_input_shape[0][-1]
        self.q_dims, self.v_dims, self.k_dims = [self.dims // self.n_heads] * 3 # could be hyperparameters instead
        self.q_linear = keras.layers.Conv1D(self.n_heads * self.q_dims, kernel_size=1, use_bias=False)
        self.v_linear = keras.layers.Conv1D(self.n_heads * self.v_dims, kernel_size=1, use_bias=False)
        self.k_linear = keras.layers.Conv1D(self.n_heads * self.k_dims, kernel_size=1, use_bias=False)
        self.attention = keras.layers.Attention(causal=self.causal, use_scale=self.use_scale)
        self.out_linear = keras.layers.Conv1D(self.dims, kernel_size=1, use_bias=False)
        super().build(batch_input_shape)
    def _multi_head_linear(self, inputs, linear):
        shape = K.concatenate([K.shape(inputs)[:-1], [self.n_heads, -1]])
        projected = K.reshape(linear(inputs), shape)
        perm = K.permute_dimensions(projected, [0, 2, 1, 3])
        return K.reshape(perm, [shape[0] * self.n_heads, shape[1], -1])
    def call(self, inputs):
        q = inputs[0]
        v = inputs[1]
        k = inputs[2] if len(inputs) > 2 else v
        shape = K.shape(q)
        q_proj = self._multi_head_linear(q, self.q_linear)
        v_proj = self._multi_head_linear(v, self.v_linear)
        k_proj = self._multi_head_linear(k, self.k_linear)
        multi_attended = self.attention([q_proj, v_proj, k_proj])
        shape_attended = K.shape(multi_attended)
        reshaped_attended = K.reshape(multi_attended, [shape[0], self.n_heads, shape_attended[1], shape_attended[2]])
        perm = K.permute_dimensions(reshaped_attended, [0, 2, 1, 3])
        concat = K.reshape(perm, [shape[0], shape_attended[1], -1])
        return self.out_linear(concat)

In [ ]:
Q = np.random.rand(2, 50, 512)
V = np.random.rand(2, 80, 512)
multi_attn = MultiHeadAttention(8)
multi_attn([Q, V]).shape